## Ensembling best 2 models:

Thanks to [Abhishek Thakur](https://www.kaggle.com/abhishek) and [Chris Deotte](https://www.kaggle.com/cdeotte) for their great notebooks. Much to learn from the 4x Grandmasters. 
I have ensembled their best public models here.

#### Links to the original notebooks:

<ul>
    <li> <a href="https://www.kaggle.com/cdeotte/rapids-svr-boost-17-8">RAPIDS SVR Boost - [17.8]</a></li>
    <li> <a href="https://www.kaggle.com/abhishek/tez-pawpular-swin-ference">tez: pawpular swin-ference</a></li>
</ul>

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
%run ../input/scriptspaw/rapids_svr_boost_17_8.py

In [ ]:
%run ../input/scriptspaw/tez_pawpular_tuning.py

In [ ]:
swin = np.load('swin.npy')
svr = np.load('rapids_svr.npy')

In [ ]:
import sys
sys.path.append('../input/timm-pytorch-image-models/pytorch-image-models-master')
from timm import create_model
from fastai.vision.all import *


In [ ]:
set_seed(999, reproducible=True)
BATCH_SIZE = 32
dataset_path = Path('../input/petfinder-pawpularity-score/')
dataset_path.ls()
train_df = pd.read_csv(dataset_path/'train.csv')
train_df.head()

In [ ]:
train_df['path'] = train_df['Id'].map(lambda x:str(dataset_path/'train'/x)+'.jpg')
train_df = train_df.drop(columns=['Id'])
train_df = train_df.sample(frac=1).reset_index(drop=True) #shuffle dataframe
train_df.head()

In [ ]:
len_df = len(train_df)
print(f"There are {len_df} images")

In [ ]:
train_df['Pawpularity'].hist(figsize = (10, 5))
print(f"The mean Pawpularity score is {train_df['Pawpularity'].mean()}")
print(f"The median Pawpularity score is {train_df['Pawpularity'].median()}")
print(f"The standard deviation of the Pawpularity score is {train_df['Pawpularity'].std()}")
print(f"There are {len(train_df['Pawpularity'].unique())} unique values of Pawpularity score")

In [ ]:
if not os.path.exists('/root/.cache/torch/hub/checkpoints/'):
    os.makedirs('/root/.cache/torch/hub/checkpoints/')
!cp '../input/swin-transformer/swin_large_patch4_window7_224_22kto1k.pth' '/root/.cache/torch/hub/checkpoints/swin_large_patch4_window7_224_22kto1k.pth'

In [ ]:
seed=999
set_seed(seed, reproducible=True)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.backends.cudnn.deterministic = True
torch.use_deterministic_algorithms = True

In [ ]:
#learn = load_learner('../input/swin-paw/models/model_fold_9.pkl.pth', cpu=False)

In [ ]:
#learn

In [ ]:
data=pd.read_csv('../input/swin-paw/submission.csv')

In [ ]:
data

In [ ]:
data_2=pd.read_csv('../input/ensemble-2/submission.csv')

In [ ]:
type(data_2.loc[0,'Pawpularity'])

In [ ]:
pred1=data['Pawpularity'].to_numpy(dtype='float64')

In [ ]:
pred_2=data_2['Pawpularity'].to_numpy(dtype='float64')

In [ ]:
pred_2

In [ ]:
type(pred1[0])

In [ ]:
score = (swin+svr+pred1+pred_2)/4

In [ ]:
test = pd.read_csv('../input/petfinder-pawpularity-score/test.csv')

test['Pawpularity'] = score
test[['Id', 'Pawpularity']].to_csv('submission.csv', index=False)

In [ ]:
test

In [ ]:
test[['Id', 'Pawpularity']].head()

#### END